## 1. Cleaning deeper

In [5]:
import pandas as pd
import numpy as np

#for sql handling
import psycopg2
import sql
from sql import engine
from sql import get_data


## Bring data from DBeaver to clean and make one single table

In [7]:
competitors_news = get_data("SELECT * FROM competitors_news")
competitors_news.head(1)

/opt/homebrew/Caskroom/miniconda/base/envs/sql-practice/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,id,unique_id,title_en,body_en,published_at,language,keywords,links.permalink,source.id,source.name,...,author.id,author.name,sentiment.body.polarity,sentiment.body.score,sentiment.title.polarity,sentiment.title.score,category_id,category_label,category_taxonomy,competitor
0,1d785dd2ea4edd43423063e700d88ff7,2022-03-18T07:48:04Z_ST Engineering completes ...,ST Engineering completes US$2.7 bil TransCore ...,​​ST Engineering completes US $2.7 bil TransCo...,2022-03-18T07:48:04Z,en,"{TransCore,""ST Engineering"",""​​ST Engineering""...",https://www.theedgesingapore.com/news/ma/st-en...,185022,"Edge, The (Singapore)",...,14463906,,positive,0.56,neutral,0.84,IAB9,Hobbies & Interests,iab-qag,ST Aerospace


In [8]:
competitors_news.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4804 entries, 0 to 4803
Data columns (total 21 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   id                        4804 non-null   object 
 1   unique_id                 4804 non-null   object 
 2   title_en                  4804 non-null   object 
 3   body_en                   4804 non-null   object 
 4   published_at              4804 non-null   object 
 5   language                  4804 non-null   object 
 6   keywords                  4804 non-null   object 
 7   links.permalink           4804 non-null   object 
 8   source.id                 4804 non-null   int64  
 9   source.name               4804 non-null   object 
 10  source_country            4295 non-null   object 
 11  author.id                 4804 non-null   int64  
 12  author.name               4804 non-null   object 
 13  sentiment.body.polarity   4804 non-null   object 
 14  sentimen

## Making articles unique by checking title and body

In [13]:
# function to make id unique with hash

import hashlib

def make_id(competitors_df):
        #new series
        title_id = pd.Series([], dtype=pd.StringDtype())
        body_id = pd.Series([], dtype=pd.StringDtype())
        
        for row in range(len(competitors_df)):

            #convert to bytes
            print(competitors_df['title_en'][row])
            str_title = bytes(competitors_df['title_en'][row], "utf-8")
            str_body = bytes(competitors_df['body_en'][row], "utf-8")
            
            # with hash make id
            title_unique_id = hashlib.md5(str_title).hexdigest()
            print('title id ', title_unique_id)
            body_unique_id = hashlib.md5(str_body).hexdigest()
            print('body id ',  body_unique_id)

            # append values to series
            title_id[row] = title_unique_id
            body_id[row] = body_unique_id
            

        # insert series in dataframe
        competitors_df.insert(1, 'title_id_unique', title_id)
        competitors_df.insert(2, 'body_id_unique', body_id)
        return competitors_df

In [14]:
make_id(competitors_news)

ST Engineering completes US$2.7 bil TransCore acquisition
title id  c2c34045d1b8f681dbe890151e0740ea
body id  0c851c8bdcc09488684e2ebfb549c50c
Delta TechOps, Asiana Airlines enter engine maintenance agreement
title id  416f933850532c8d9c6d06898603cf3a
body id  38a90cd18ae9486319f0acd8e69c5d66
SR Technics creates up to 400 new production sites
title id  fb1bb049dfe6ea0f1b301384f78ee292
body id  5f818181370b5ac1310c2879e84a25e8
UOB Kay Hian expects Singapore aviation sector to recover by end-2024, sees SIAEC as top pick to ride on recovery
title id  04e04a6c59d3c6d44ac3649446442097
body id  2e0aa8babeaf5cccb40a4c6448eb77b4
Turkish Technic reports 15.7% growth in FY2021
title id  96deb2f5e91fda7bd726176e612a2b26
body id  c277a4f82e4e89ba7fb7b8c78e491214
Turkish Technic signed new ground maintenance agreements with AMROS Global, Air Dubai and Jazeera Airways in the 'MRO Middle East 2022 Expo'.
title id  f2350e20804af195ba88eae42666da50
body id  0c8afc3b1614187311c839582aed8c84
Turkish Tech

,id,title_id_unique,body_id_unique,unique_id,title_en,body_en,published_at,language,keywords,links.permalink,...,author.id,author.name,sentiment.body.polarity,sentiment.body.score,sentiment.title.polarity,sentiment.title.score,category_id,category_label,category_taxonomy,competitor
0,1d785dd2ea4edd43423063e700d88ff7,c2c34045d1b8f681dbe890151e0740ea,0c851c8bdcc09488684e2ebfb549c50c,2022-03-18T07:48:04Z_ST Engineering completes ...,ST Engineering completes US$2.7 bil TransCore ...,​​ST Engineering completes US $2.7 bil TransCo...,2022-03-18T07:48:04Z,en,"{TransCore,""ST Engineering"",""​​ST Engineering""...",https://www.theedgesingapore.com/news/ma/st-en...,...,14463906,,positive,0.56,neutral,0.84,IAB9,Hobbies & Interests,iab-qag,ST Aerospace
1,72c273af8b9cbab6e2c6628cf0c16c6e,416f933850532c8d9c6d06898603cf3a,38a90cd18ae9486319f0acd8e69c5d66,"2022-04-27T23:51:03Z_Delta TechOps, Asiana Air...","Delta TechOps, Asiana Airlines enter engine ma...",Delta TechOps people will expertly carry out s...,2022-04-27T23:51:03Z,en,"{work,""Asiana Airlines"",engines,""Star Alliance...",https://www.publicnow.com/view/AC95E60DF724470...,...,3036148,,positive,0.99,neutral,0.96,IAB20-7,Business Travel,iab-qag,Delta
2,70a2fa43510b492f3ccbba27144ba673,fb1bb049dfe6ea0f1b301384f78ee292,5f818181370b5ac1310c2879e84a25e8,2022-04-06T12:45:48Z_SR Technics creates up to...,SR Technics creates up to 400 new production s...,"Published on 06.04.2022 - 14:27, updated 21 mi...",2022-04-06T12:45:48Z,de,"{neuen,Griechen,Vertrag,A320neo-Flugzeuge,Airb...",https://www.handelszeitung.ch/newsticker/sr-te...,...,3527086,,positive,0.96,neutral,0.75,IAB3-5,Construction,iab-qag,SR Technics
3,345636d0297fe110145c7a663c23d10b,04e04a6c59d3c6d44ac3649446442097,2e0aa8babeaf5cccb40a4c6448eb77b4,2022-03-29T12:15:01Z_UOB Kay Hian expects Sing...,UOB Kay Hian expects Singapore aviation sector...,UOB Kay Hian analyst Roy Chen has maintained h...,2022-03-29T12:15:01Z,en,"{SIAEC,Hian,pick,weight,""Kay Hian"",market,reco...",https://www.theedgesingapore.com/capital/broke...,...,14463906,,positive,0.81,neutral,0.87,IAB2-16,Off-Road Vehicles,iab-qag,SIAEC
4,0d5e899d42221ec438e901fa671ca586,96deb2f5e91fda7bd726176e612a2b26,c277a4f82e4e89ba7fb7b8c78e491214,2022-03-23T16:16:45Z_Turkish Technic reports 1...,Turkish Technic reports 15.7% growth in FY2021,"As with most areas of aviation, the MRO sector...",2022-03-23T16:16:45Z,en,"{numbers,MRO,pandemic,Boeing,""Mikail Akbulut"",...",https://www.aircraftinteriorsinternational.com...,...,3086247,Adam Gavine,positive,0.65,neutral,0.74,IAB2-2,Auto Repair,iab-qag,Turkish Technic
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4799,7632df6044e93dfd7b9bf129af6bed37,2c837ec2e2742eaa28714ac2e213f5fb,fe537a7eb558d395079fcf6689a3bad6,2022-06-03T16:19:15Z_NASA selects Axiom Space ...,NASA selects Axiom Space and Collins Aerospace...,in order to provide space for international sp...,2022-06-03T16:19:15Z,sv,"{spacewalks,""Jared Isaacman"",Wickess,KBR,Metav...",https://zephyrnet.com/sv/nasa-v%C3%A4ljer-axio...,...,26934115,Meta Time,positive,0.49,neutral,0.89,IAB15-7,Space/Astronomy,iab-qag,Raytheon Technologies
4800,8a5572ff40c0b9eb42ca3c877b04015f,a9fbd3f6b050749844c1754f7ef766d6,5bbca2138088a2005f6671a55df4561c,2022-06-03T17:54:02Z_Airplane seats 2022-2031:...,Airplane seats 2022-2031: Current and Futures ...,"and its commercial strategies, geographical de...",2022-06-03T17:54:02Z,es,"{Inversores,région,Liga,Evaluación,Informe,Ita...",https://hoyciclismo.com/asientos-de-aeronaves-...,...,26779221,Aditya Deshmukh,positive,0.68,neutral,0.95,IAB13-11,Stocks,iab-qag,Raytheon Technologies
4801,3458319e8329469fd8ef5893fdc31bd0,d225218f4f8b8a5d52214ffe1dde348d,1da8aff8bfa0c4a854d0ae6f272877ed,"2022-06-04T02:07:36Z_Running: Goodrich, Gray t...","Running: Goodrich, Gray top finishers in first...",Goodrich completed the 3.1-mile course in 18:1...,2022-06-04T02:07:36Z,en,"{mile,Rachel,Legett,""Owen Legett"",Gray,""Jeff W...",h

In [15]:
competitors_news.drop(columns=['id', 'unique_id'], axis=1, inplace=True)

In [16]:
competitors_news.head(1)

,title_id_unique,body_id_unique,title_en,body_en,published_at,language,keywords,links.permalink,source.id,source.name,...,author.id,author.name,sentiment.body.polarity,sentiment.body.score,sentiment.title.polarity,sentiment.title.score,category_id,category_label,category_taxonomy,competitor
0,c2c34045d1b8f681dbe890151e0740ea,0c851c8bdcc09488684e2ebfb549c50c,ST Engineering completes US$2.7 bil TransCore ...,​​ST Engineering completes US $2.7 bil TransCo...,2022-03-18T07:48:04Z,en,"{TransCore,""ST Engineering"",""​​ST Engineering""...",https://www.theedgesingapore.com/news/ma/st-en...,185022,"Edge, The (Singapore)",...,14463906,,positive,0.56,neutral,0.84,IAB9,Hobbies & Interests,iab-qag,ST Aerospace


In [19]:
# define function to look up keywords in body

def unique_news_upload(competitors_df):
    
    make_id(competitors_df)
    
    table_name = 'tes_news_2'
    schema = 'capstone'
    for row in range(len(competitors_df)):
        try:
            competitors_df.iloc[row:row+1, :].to_sql(name=table_name, # Name of SQL table
                            con=engine, # Engine or connection
                            if_exists='append', # change to 'append' 
                            schema=schema, # Use schmea that was defined earlier
                            index=False, # Write DataFrame index as a column
                            chunksize=1, # Specify the number of rows in each batch to be written at a time
                            method='multi') # Pass multiple values in a single INSERT clause
            print(f"row {row} of news df was inserted successfully.")
        # Error handling
        except:
            print(f"row {row} of news df was ignored.")
            

In [20]:
unique_news_upload(competitors_news)

ST Engineering completes US$2.7 bil TransCore acquisition
title id  c2c34045d1b8f681dbe890151e0740ea
body id  0c851c8bdcc09488684e2ebfb549c50c
Delta TechOps, Asiana Airlines enter engine maintenance agreement
title id  416f933850532c8d9c6d06898603cf3a
body id  38a90cd18ae9486319f0acd8e69c5d66
SR Technics creates up to 400 new production sites
title id  fb1bb049dfe6ea0f1b301384f78ee292
body id  5f818181370b5ac1310c2879e84a25e8
UOB Kay Hian expects Singapore aviation sector to recover by end-2024, sees SIAEC as top pick to ride on recovery
title id  04e04a6c59d3c6d44ac3649446442097
body id  2e0aa8babeaf5cccb40a4c6448eb77b4
Turkish Technic reports 15.7% growth in FY2021
title id  96deb2f5e91fda7bd726176e612a2b26
body id  c277a4f82e4e89ba7fb7b8c78e491214
Turkish Technic signed new ground maintenance agreements with AMROS Global, Air Dubai and Jazeera Airways in the 'MRO Middle East 2022 Expo'.
title id  f2350e20804af195ba88eae42666da50
body id  0c8afc3b1614187311c839582aed8c84
Turkish Tech

ValueError: cannot insert title_id_unique, already exists